## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [3]:
NAME = "Yfantidis Dimitrios"
AEM = "3938"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [4]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [13]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x282dd914550>)

In [5]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

In [ ]:
!pip install -U imbalanced-learn

The following code will reduce the number of instances, dealing with the small imbalance of the dataset, as well as reducing the size of the dataset!

In [6]:
from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, RandomUnderSampler

ncr = NeighbourhoodCleaningRule()
X_res, y_res = ncr.fit_resample(X, y)
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res))
X = X_res
y = y_res 

Resampled dataset shape Counter({0: 1687, 1: 1687})


## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **5-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. 

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses two **simple** estimators/classifiers. Test both soft and hard voting and report the results. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   KNN Models  

In [7]:
### BEGIN SOLUTION
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
from statistics import mean


# USE RANDOM STATE!
cls1 = KNeighborsClassifier(n_neighbors=5)
cls2 = LogisticRegression()
# cls2 = DecisionTreeClassifier(max_depth=7, criterion='gini', random_state=RANDOM_STATE)
soft_vcls = VotingClassifier([('KNN', cls1), ('Log Regression', cls2)], voting='soft')
hard_vcls = VotingClassifier([('KNN', cls1), ('Log Regression', cls2)], voting='hard')

svlcs_scores = cross_validate(soft_vcls, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
s_avg_fmeasure = mean(svlcs_scores['test_f1_weighted'])
s_avg_accuracy = mean(svlcs_scores['test_balanced_accuracy'])

hvlcs_scores = cross_validate(hard_vcls, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
h_avg_fmeasure = mean(hvlcs_scores['test_f1_weighted'])
h_avg_accuracy = mean(hvlcs_scores['test_balanced_accuracy'])
### END SOLUTION

print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('KNN', KNeighborsClassifier()),
                             ('Log Regression', LogisticRegression())],
                 voting='soft')
F1 Weighted-Score: 0.8974 & Balanced Accuracy: 0.8974
Classifier:
VotingClassifier(estimators=[('KNN', KNeighborsClassifier()),
                             ('Log Regression', LogisticRegression())])
F1 Weighted-Score: 0.8587 & Balanced Accuracy: 0.8601


For both soft/hard voting classifiers the F1 weighted score should be above 0.74 and 0.79, respectively, and for balanced accuracy 0.74 and 0.80. Remember! This should be the average performance of each fold, as measured through cross-validation with 5 folds!

### 1.2 Randomization

You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements. Use the same cross-validation approach as before! 

In [8]:
### BEGIN SOLUTION
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier


ens1 = BaggingClassifier(
    DecisionTreeClassifier(max_depth=5, criterion='gini', random_state=RANDOM_STATE), 
    n_estimators=20, 
    random_state=RANDOM_STATE
)

ens2 = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=5, criterion='gini', random_state=RANDOM_STATE), 
    n_estimators=20,
    random_state=RANDOM_STATE
)

ens3 = RandomForestClassifier(
    criterion='gini',
    n_estimators=20, 
    random_state=RANDOM_STATE
)

tree = DecisionTreeClassifier(max_depth=6, criterion='gini', random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}

cv1 = cross_validate(ens1, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
cv2 = cross_validate(ens2, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
cv3 = cross_validate(ens3, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
cv_tree = cross_validate(tree, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)

names = ['Simple Decision Tree', 'Bagging Classifier', 'AdaBoost Classifier', 'Random Forest']
cv = [cv_tree, cv1, cv2, cv3]

for i in range(4):
    f_measures[names[i]] = mean(cv[i]['test_f1_weighted'])
    accuracies[names[i]] = mean(cv[i]['test_balanced_accuracy'])
### END SOLUTION

print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier: {} -  F1 Weighted: {}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier: {} -  BalancedAccuracy: {}".format(name,round(score,4)))

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                   random_state=42),
                  n_estimators=20, random_state=42)
AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                    random_state=42),
                   n_estimators=20, random_state=42)
RandomForestClassifier(n_estimators=20, random_state=42)
DecisionTreeClassifier(max_depth=6, random_state=42)
Classifier: Simple Decision Tree -  F1 Weighted: 0.7501
Classifier: Bagging Classifier -  F1 Weighted: 0.8038
Classifier: AdaBoost Classifier -  F1 Weighted: 0.7672
Classifier: Random Forest -  F1 Weighted: 0.8366
Classifier: Simple Decision Tree -  BalancedAccuracy: 0.7504
Classifier: Bagging Classifier -  BalancedAccuracy: 0.8041
Classifier: AdaBoost Classifier -  BalancedAccuracy: 0.7673
Classifier: Random Forest -  BalancedAccuracy: 0.8367


### 1.3 Question

Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

> Yes, increasing the number of estimators in a `Bagging Classifier` can indeed lead to a significant increase in training time. This is because each estimator is  
> trained independently, and the computational complexity grows linearly with the number of estimators. This problem persists in other ensembles like `Random Forest`  
> and `Adaptive Boosting Classifier`.  
> To combat this problem, we utilise `hardware parallelization`, in the case of Bagging and Random Forest, as every tree is trained independently. This is not the  
> case for Boosting as each tree/stump is trained based on the training results of the ones before it.

## 2.0 Creating the best classifier ##
In the second part of this assignment, we will try to train the best classifier, as well as to evaluate it using stratified cross valdiation.

### 2.1 Good Performing Ensemble

In this part of the assignment you are asked to train a good performing ensemble, that is able to be used in a production environment! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy, using 10-fold stratified cross validation, of your final classifier. Can you achieve a balanced accuracy over 88%, while keeping the training time low? (Tip 1: You can even use a model from the previous parts, but you are advised to test additional configurations, and ensemble architectures, Tip 2: If you try a lot of models/ensembles/configurations or even grid searches, in your answer leave only the classifier you selected as the best!)

In [12]:
### BEGIN SOLUTION
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold


best_cls = XGBClassifier(n_estimators=200, tree_method='gpu_hist').fit(X, y)
cv_cls = cross_validate(
    best_cls, X, y, 
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE), 
    scoring=('f1_weighted', 'balanced_accuracy')
)

best_fmeasure = mean(cv_cls['test_f1_weighted'])
best_accuracy = mean(cv_cls['test_balanced_accuracy'])
### END SOLUTION

print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score: 0.8917998396993587 & Balanced Accuracy: 0.8918234009580163


> Many models were tested. From them, `XGBoost Classifier` is the only powerful enough to guarantee greater scoring than 88% **consistently**, given the fact that the fourth (predifined) cell doesn't use the `RANDOM_STATE` seed for loading the database. It is also extremely powerful, so powerful that it caused my computer a blue screen/out of memory error.  
> That happened because `XGBClassifier` uses a huge amount of VRAM and, thus, parallelization on all cores is prohibitive.  
> For that reason CV is executed in a serialized manner.

### 2.2 Question
 What other ensemble architectures you tried, and why you did not choose them as your final classifier?

> I also used `Grid Searches` and then `Randomized Searches` to find the best hyperparameters for the `Random Forest Ensemble`, as it outperforms AdaBoost and Bagging. This process was taking too much time and processing power, so I started trying inserting hyperparameters manually. In the end, I found a combination of parameters that scored above 88% in both F1 wighted and Balanced Accuracy but, as mentioned above, the cell that loads the database doesn't use the `RANDOM_STATE` variable. As a consequence, the program was not consistent in scoring above the threshold in different sessions. `XGBoost` proved to be the only consistent enough.

### 2.3 Setup the Final Classifier
Finally, in this last cell, set the cls variable to either the best model as occured by the stratified cross_validation, or choose to retrain your classifier in the whole dataset (X, y). There is no correct answer, but try to explain your choice. Then, save your model using pickle and upload it with your submission to e-learning!

In [11]:
import pickle

### BEGIN SOLUTION
cls = best_cls

# save with pickle
file_name = "best_classifier.pkl"
pickle.dump(cls, open(file_name, "wb"))
### END SOLUTION


# load
cls = pickle.load(open(file_name, "rb"))

test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

# We are going to run the following code
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both metrics should aim above 82%! This is going to be tested by us! Make sure your cross validation or your retrained model achieves high balanced accuracy and f1_score (based on 2.1) (more than 88%) as it should achieve at least 82% in our unknown test set!


Please provide your feedback regarding this project! Did you enjoy it? 

In [ ]:
# YOUR ANSWER HERE